# THIS IS AN ATTEMPT OF AUTOMATING STOCKPICK WITH FUNDAMENTAL ANALYSIS

# METHOD USED:
1.Piotroski-F Score
2.Dividend Yield
3.Growth
4.Industry comparison

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import yahoo_fin.stock_info as yf

In [2]:
balance_sheet=[]
income_statement=[]
cfs=[]
years=[]
profitability_score = 0
leverage_score= 0
operating_efficiency_score = 0

USE THIS IF U WANT TO SEARCH FOR BEST BANK INDUSTRY STOCK

In [3]:
bank_data=open('banks.txt', 'r')
bank_content=bank_data.read()
banks=bank_content.split(',')
bank_data.close()
banks

['BBRI.JK',
 'BBNI.JK',
 'BBCA.JK',
 'BMRI.JK',
 'BRIS.JK',
 'BTPS.JK',
 'BNGA.JK',
 'BTPN.JK',
 'BBTN.JK',
 'BDMN.JK',
 'BNLI.JK',
 'BJBR.JK',
 'BJTM.JK']

In [4]:
bank_tickers= []
for i in banks:
    bank_tickers.append(i)
bank_tickers

['BBRI.JK',
 'BBNI.JK',
 'BBCA.JK',
 'BMRI.JK',
 'BRIS.JK',
 'BTPS.JK',
 'BNGA.JK',
 'BTPN.JK',
 'BBTN.JK',
 'BDMN.JK',
 'BNLI.JK',
 'BJBR.JK',
 'BJTM.JK']

RUN THIS FOR INFRASTRUCTURE STOCKS

In [3]:
infra_data=open('infra.txt', 'r')
infra_content=infra_data.read()
infrastructures=infra_content.split(',')
infra_data.close()
infrastructures

['ADHI.JK',
 'CENT.JK',
 'EXCL.JK',
 'IPCM.JK',
 'IPCC.JK',
 'ISAT.JK',
 'JSMR.JK',
 'PTPP.JK',
 'TBIG.JK',
 'TLKM.JK',
 'WIKA.JK',
 'WSKT.JK',
 'TOWR.JK',
 'SSIA.JK']

In [4]:
infra_tickers= []
for i in infrastructures:
    infra_tickers.append(i)
infra_tickers

['ADHI.JK',
 'CENT.JK',
 'EXCL.JK',
 'IPCM.JK',
 'IPCC.JK',
 'ISAT.JK',
 'JSMR.JK',
 'PTPP.JK',
 'TBIG.JK',
 'TLKM.JK',
 'WIKA.JK',
 'WSKT.JK',
 'TOWR.JK',
 'SSIA.JK']

In [5]:
def get_data(ticker):
    global balance_sheet
    global income_statement
    global cfs
    global years
    
    balance_sheet = yf.get_balance_sheet(ticker)
    income_statement = yf.get_income_statement(ticker)
    cfs = yf.get_cash_flow(ticker)
    years=balance_sheet.columns

In [6]:
def profitability():
    global profitability_score
    #Scores number 1 and 2 for Net Income
    net_income = income_statement[years[0]]['netIncome']
    net_income_py = income_statement[years[1]]['netIncome']
    ni_score = 1 if net_income > 0 else 0
    ni_score_2 = 1 if net_income > net_income_py else 0
    
    #Score number 3 for Operating Cashflow
    op_cf = cfs[years[0]]['totalCashFromOperatingActivities']
    op_cf_score = 1 if op_cf > 0 else 0
    
    #Score number 4 for RoA
    avg_assets=(balance_sheet[years[0]]['totalAssets'] + 
                balance_sheet[years[1]]['totalAssets']) / 2
    avg_assets_py=(balance_sheet[years[1]]['totalAssets'] + 
                balance_sheet[years[2]]['totalAssets']) / 2
    
    RoA = net_income/avg_assets
    RoA_py = net_income_py/avg_assets_py
    RoA_score = 1 if RoA > RoA_py else 0
    
    #Score number 5 for Accruals
    total_assets = balance_sheet[years[0]]['totalAssets']
    accruals = op_cf/total_assets - RoA
    ac_score = 1 if accruals > 0 else 0
    
    profitability_score = ni_score + ni_score_2 + op_cf_score + RoA_score + ac_score
    return profitability_score
    

In [7]:
def leverage():
    global leverage_score
    
    #Score number 6 for long term debt ratio
    try:
        lt_debt = balance_sheet[years[0]]['longTermDebt']
        total_assets = balance_sheet[years[0]]['totalAssets']
        debt_ratio = lt_debt / total_assets
        debt_ratio_score = 1 if debt_ratio < 0.4 else 0
    except:
        debt_ratio_score =1 
        
    #Score number 7 for current ratio
    current_assets = balance_sheet[years[0]]['totalCurrentAssets']
    current_liab = balance_sheet[years[0]]['totalCurrentLiabilities']
    current_ratio = current_assets / current_liab
    current_ratio_score = 1 if current_ratio > 1 else 0
    
    leverage_score = debt_ratio_score + current_ratio_score
    return leverage_score

In [8]:
def operating_efficiency():
    global operating_efficiency_score
    
    #Score number 8 for change in gross margin
    gp = income_statement[years[0]]['grossProfit']
    gp_py = income_statement[years[1]]['grossProfit']
    revenue = income_statement[years[0]]['totalRevenue']
    revenue_py = income_statement[years[1]]['totalRevenue']
    try:
        gm = gp /revenue
        gm_py = gp/ revenue_py
        gross_margin_score = 1 if gm > gm_py else 0
    except:
        gross_margin_score = 0
        
    #Score number 9 for asset turnover (revenue/avg assets), how much revenue per assets
    avg_assets=(balance_sheet[years[0]]['totalAssets'] + 
                balance_sheet[years[1]]['totalAssets']) / 2
    avg_assets_py=(balance_sheet[years[1]]['totalAssets'] + 
                balance_sheet[years[2]]['totalAssets']) / 2
    try:
        assets_turnover = revenue/avg_assets
        assets_turnover_py = revenue_py / avg_assets_py
        at_score = 1 if assets_turnover > assets_turnover_py else 0
    except: 
        at_score = 0
        
    operating_efficiency_score = gross_margin_score + at_score
    return operating_efficiency_score

In [9]:
piotroski = []
ticker_profitability = []
ticker_leverage_data =[]
ticker_operating_eff = []
"""
for ticker in sector_tickers
    bank: bank_tickers
    infrastructures: infra_tickers
"""

for ticker in infra_tickers:
    try:
        get_data(ticker)
        print(ticker)
        p_score = profitability()
        l_score = leverage()
        o_score = operating_efficiency()
        total_score = p_score + l_score + o_score
        piotroski.append(total_score)
        ticker_profitability.append(p_score)
        ticker_leverage_data.append(l_score)
        ticker_operating_eff.append(o_score)
    except:
        print ('something went wrong for ' + ticker)



ADHI.JK
CENT.JK
EXCL.JK
IPCM.JK
IPCC.JK
ISAT.JK
JSMR.JK
PTPP.JK
TBIG.JK
TLKM.JK
WIKA.JK
WSKT.JK
TOWR.JK
SSIA.JK


In [10]:
piotroski

[6, 5, 4, 8, 5, 3, 6, 5, 5, 5, 7, 6, 4, 6]

In [11]:
#remember to change the tickers below according to the analyzed sector
dict = {'ticker': infra_tickers, 'piotroski_score': piotroski, 'profitability': ticker_profitability,
       'leverage': ticker_leverage_data, 'operating_efficiency': ticker_operating_eff}

In [12]:
df = pd.DataFrame(dict)
df

,ticker,piotroski_score,profitability,leverage,operating_efficiency
0,ADHI.JK,6,3,2,1
1,CENT.JK,5,3,2,0
2,EXCL.JK,4,3,1,0
3,IPCM.JK,8,5,2,1
4,IPCC.JK,5,3,2,0
5,ISAT.JK,3,2,1,0
6,JSMR.JK,6,4,1,1
7,PTPP.JK,5,2,2,1
8,TBIG.JK,5,5,0,0
9,TLKM.JK,5,4,1,0


In [13]:
df.loc[df['piotroski_score'].idxmax()]

ticker                  IPCM.JK
piotroski_score               8
profitability                 5
leverage                      2
operating_efficiency          1
Name: 3, dtype: object